In [1]:
import numpy as np
import pandas as pd

In [2]:
import pickle

with open("train_qa.txt", "rb") as fp:   
    train_data =  pickle.load(fp)

with open("test_qa.txt", "rb") as fp:   
    test_data =  pickle.load(fp)

# Creating a Dictionary of words
This Dictionary will be used in training the model

In [3]:
words = set()
all_data = train_data + test_data

In [4]:
for i, j, k in all_data:
    words = words.union(set(i))
    words = words.union(set(j))
    
words.add('yes')
words.add('no')

In [6]:
words

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'no',
 'office',
 'picked',
 'put',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'yes'}

In [7]:
words_len = len(words) + 1 # Adding 1 space extra for pad_sequences - Reserving 0 

# Creating a Function for Vectorizing the Data

In [9]:
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters = [])
tokenizer.fit_on_texts(words)

In [10]:
max_story_len = max([len(data[0]) for data in all_data])
max_query_len = max([len(data[1]) for data in all_data])

In [11]:
def vectorize_data(data, word_index=tokenizer.word_index, max_story_len=max_story_len,max_query_len=max_query_len):
    X = []
    Q = []
    A = []
    
    
    for story, query, answer in data:
        
        x = [word_index[word.lower()] for word in story]
        q = [word_index[word.lower()] for word in query]
        a = np.zeros(len(word_index) + 1)

        a[word_index[answer]] = 1
        
        X.append(x)
        Q.append(q)
        A.append(a)
        
    return (pad_sequences(X, maxlen=max_story_len),pad_sequences(Q, maxlen=max_query_len), np.array(A))

In [12]:
train_input, train_query, train_answer = vectorize_data(train_data)
test_input, test_query, test_answer = vectorize_data(test_data)

# Building the Neural Network

In [13]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Input, Activation, Dropout
from tensorflow.keras.layers import add, dot, concatenate, Permute, Dense
from tensorflow.keras.layers import LSTM

In [15]:
input_sequence = Input(shape = (max_story_len,),)
input_query = Input(shape = (max_query_len,),)

In [16]:
input_encoder_memory = Sequential()
input_encoder_memory.add(Embedding(input_dim = words_len, output_dim = 64))
input_encoder_memory.add(Dropout(0.3))

input_encoder_continuous = Sequential()
input_encoder_continuous.add(Embedding(input_dim = words_len, output_dim = max_query_len))
input_encoder_continuous.add(Dropout(0.3))

question_encoder = Sequential()
question_encoder.add(Embedding(input_dim = words_len, output_dim = 64, input_length = max_query_len))
question_encoder.add(Dropout(0.3))

In [17]:
input_encoded_memory = input_encoder_memory(input_sequence)
input_encoded_continuous = input_encoder_continuous(input_sequence)
question_encoded = question_encoder(input_query)

In [18]:
match = dot([input_encoded_memory, question_encoded], axes = (2,2))
match = Activation('softmax')(match)

response = add([match, input_encoded_continuous])
response = Permute((2,1))(response)

answer = concatenate([response, question_encoded])

In [19]:
answer = LSTM(32)(answer)
answer = Dropout(0.5)(answer)
answer = Dense(words_len)(answer)
answer = Activation('softmax')(answer)

In [21]:
model = Model([input_sequence, input_query], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 156)]                0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 6)]                  0         []                            
                                                                                                  
 sequential (Sequential)     (None, None, 64)             2432      ['input_3[0][0]']             
                                                                                                  
 sequential_2 (Sequential)   (None, 6, 64)                2432      ['input_4[0][0]']             
                                                                                            

In [23]:
history = model.fit([train_input, train_query], train_answer, batch_size=32, epochs=150,validation_data=([test_input, test_query], test_answer))

Epoch 1/150


313/313 [==============================] - 27s 33ms/step - loss: 0.8483 - accuracy: 0.5018 - val_loss: 0.6954 - val_accuracy: 0.5030
Epoch 2/150
313/313 [==============================] - 7s 23ms/step - loss: 0.7027 - accuracy: 0.5010 - val_loss: 0.6934 - val_accuracy: 0.5030
Epoch 3/150
313/313 [==============================] - 7s 22ms/step - loss: 0.6973 - accuracy: 0.4966 - val_loss: 0.6957 - val_accuracy: 0.5030
Epoch 4/150
313/313 [==============================] - 7s 23ms/step - loss: 0.6963 - accuracy: 0.4961 - val_loss: 0.6979 - val_accuracy: 0.4970
Epoch 5/150
313/313 [==============================] - 7s 23ms/step - loss: 0.6957 - accuracy: 0.5047 - val_loss: 0.6964 - val_accuracy: 0.4970
Epoch 6/150
313/313 [==============================] - 7s 23ms/step - loss: 0.6954 - accuracy: 0.5009 - val_loss: 0.6933 - val_accuracy: 0.4970
Epoch 7/150
313/313 [==============================] - 7s 23ms/step - loss: 0.6954 - accuracy: 0.5012 - val_loss: 0.6933 - val_accura

In [24]:
filename = 'chatbot_using_rnn.keras'
model.save(filename)